In [1]:
import os
import time
import math
import random
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from scipy.stats import ks_2samp
from sdv.metadata import MultiTableMetadata
from sdv.evaluation.single_table import evaluate_quality as st_evaluate_quality
from sdv.evaluation.single_table import run_diagnostic as st_run_diagnostic
from sdv.evaluation.multi_table import evaluate_quality as mt_evaluate_quality
from sdv.evaluation.multi_table import run_diagnostic as mt_run_diagnostic

# Load Processed Data From Generation Stage

In [2]:
with open('pkl/real_data_collection.pkl', 'rb') as f:
    real_data_collection = pickle.load(f)

In [3]:
with open('pkl/synthetic_data_collection.pkl', 'rb') as f:
    synthetic_data_collection = pickle.load(f)

In [4]:
with open('pkl/sdv_metadata.pkl', 'rb') as f:
    sdv_metadata = pickle.load(f)

In [5]:
real_data_collection

{'agency':     agency_id  agency_url  agency_timezone  agency_lang  agency_phone
 0           0           0                0            0             0
 1           1           0                0            0             0
 2           2           0                0            0             0
 3           3           0                0            0             0
 4           4           0                0            0             0
 5           5           0                0            0             0
 6           6           0                0            0             0
 7           7           0                0            0             0
 8           8           0                0            0             0
 9           9           0                0            0             0
 10         10           0                0            0             0
 11         11           0                0            0             0
 12         12           0                0            0           

In [6]:
synthetic_data_collection

{'agency':     agency_id  agency_url  agency_timezone  agency_lang  agency_phone
 0           0           0                0            0             0
 1           1           0                0            0             0
 2           2           0                0            0             0
 3           3           0                0            0             0
 4           4           0                0            0             0
 5           5           0                0            0             0
 6           6           0                0            0             0
 7           7           0                0            0             0
 8           8           0                0            0             0
 9           9           0                0            0             0
 10         10           0                0            0             0
 11         11           0                0            0             0
 12         12           0                0            0           

# Metrics

### Cosine Similarity Test

In [7]:
def cos_test(df1, df2):
    cos_sim = metrics.pairwise.cosine_similarity(df1.values.T, df2.values.T)
    mean_cos_sim = np.mean(cos_sim)
    return mean_cos_sim

In [8]:
def batch_cos_test(fake_collection, real_collection):
    test_dict = {}
    for df_name in fake_collection.keys():
        mean_cos_sim = cos_test(real_collection[df_name], fake_collection[df_name])
        test_dict[df_name] = str(round((1-mean_cos_sim)*100, 1)) + '% Similar'
    return test_dict

In [9]:
%time batch_cos_test(synthetic_data_collection, real_data_collection)

CPU times: total: 1.64 s
Wall time: 211 ms


{'agency': '96.0% Similar',
 'calendar': '79.7% Similar',
 'calendar_dates': '7.9% Similar',
 'routes': '70.1% Similar',
 'stops': '81.1% Similar',
 'stop_times': '56.8% Similar',
 'trips': '62.2% Similar'}

### Inverse KS (Kolmogorov-Smirnov) Test

In [10]:
# def batch_ks_test(fake_collection, real_collection):
#     stats_dict = {}
#     for df_name, fake_df in fake_collection.items():
#         stats_list = []
#         p_val_list = []
#         test_dict = {}
#         for col in fake_df.columns:
#             stats, p_val = ks_2samp(fake_df[col], real_collection[df_name][col], alternative='two-sided', method='auto')
#             stats_list.append(stats)
#             p_val_list.append(p_val)
#         test_dict['stats'] = np.mean(stats_list)
#         test_dict['p_value'] = np.mean(p_val_list)
#         stats_dict[df_name] = test_dict
#     return stats_dict

In [11]:
def ks_test(df1, df2):
    ks_stats = []
    # wasserstein_distances = []
    p_values = []
    for column in df1.columns:
        ks_stat, ks_p_value = ks_2samp(df1[column], df2[column])
        ks_stats.append(ks_stat)
        p_values.append(ks_p_value)
        # w_dist = wasserstein_distance(df1[column], df2[column])
        # wasserstein_distances.append(w_dist)
    mean_ks_stat = np.mean(ks_stats)
    # mean_wasserstein_distance = np.mean(wasserstein_distances)
    mean_p_value = np.mean(p_values)
    return mean_ks_stat, mean_p_value

In [12]:
def batch_ks_test(collection1, collection2):
    stats_dict = {}
    for df_name in collection1:
        ks_results, p_value = ks_test(collection1[df_name], collection2[df_name])
        stats_dict[df_name] = str(round((1-ks_results)*100, 1)) + '% similar in distribution'
    return stats_dict

In [13]:
%time batch_ks_test(synthetic_data_collection, real_data_collection)

CPU times: total: 3.17 s
Wall time: 2.19 s


{'agency': '100.0% similar in distribution',
 'calendar': '56.9% similar in distribution',
 'calendar_dates': '69.1% similar in distribution',
 'routes': '55.8% similar in distribution',
 'stops': '52.4% similar in distribution',
 'stop_times': '68.4% similar in distribution',
 'trips': '71.6% similar in distribution'}

# SDV Individual Table Metrics

In [14]:
individual_report_collection = {}
for df_name in real_data_collection.keys():
    print(f'[{df_name}]:')
    individual_report_collection[df_name] = st_evaluate_quality(
        real_data=real_data_collection[df_name],
        synthetic_data=synthetic_data_collection[df_name],
        metadata=sdv_metadata.tables[df_name])
    print('--------------------')
    print()

[agency]:


Creating report: 100%|███████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 60.59it/s]



Overall Quality Score: 100.0%

Properties:
Column Shapes: 100.0%
Column Pair Trends: 100.0%
--------------------

[calendar]:


Creating report: 100%|███████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 18.26it/s]



Overall Quality Score: 46.48%

Properties:
Column Shapes: 52.07%
Column Pair Trends: 40.9%
--------------------

[calendar_dates]:


...


Overall Quality Score: 41.27%

Properties:
Column Shapes: 45.46%
Column Pair Trends: 37.09%
--------------------

[stop_times]:


Creating report: 100%|███████████████████████████████████████████████████████████████████| 4/4 [00:12<00:00,  3.22s/it]



Overall Quality Score: 51.73%

Properties:
Column Shapes: 59.71%
Column Pair Trends: 43.76%
--------------------

[trips]:


Creating report: 100%|███████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  9.52it/s]


Overall Quality Score: 56.72%

Properties:
Column Shapes: 71.15%
Column Pair Trends: 42.29%
--------------------



In [15]:
individual_diagnostic_collection = {}
for df_name in real_data_collection.keys():
    print(f'[{df_name}]:')
    individual_diagnostic_collection[df_name] = st_run_diagnostic(
        real_data=real_data_collection[df_name],
        synthetic_data=synthetic_data_collection[df_name],
        metadata=sdv_metadata.tables[df_name])
    print('--------------------')
    print()

[agency]:


Creating report: 100%|██████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 153.81it/s]
C:\programs\conda\envs\tfgpu\lib\site-packages\sdmetrics\reports\single_table\diagnostic_report.py:214: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if pd.Series(errors).notna().sum() > 0:



DiagnosticResults:

SUCCESS:
✓ The synthetic data covers over 90% of the categories present in the real data

DANGER:
x More than 50% of the synthetic rows are copies of the real data
--------------------

[calendar]:


Creating report: 100%|███████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 12.30it/s]



DiagnosticResults:

! The synthetic data is missing more than 10% of the categories present in the real data
! More than 10% the synthetic data does not follow the min/max boundaries set by the real data

DANGER:
x The synthetic data is missing more than 50% of the numerical ranges present in the real data
x More than 50% of the synthetic rows are copies of the real data
--------------------

[calendar_dates]:


...

Creating report: 100%|███████████████████████████████████████████████████████████████████| 4/4 [02:03<00:00, 30.97s/it]



DiagnosticResults:

SUCCESS:
✓ Over 90% of the synthetic rows are not copies of the real data
✓ The synthetic data follows over 90% of the min/max boundaries set by the real data

! The synthetic data is missing more than 10% of the categories present in the real data

DANGER:
x The synthetic data is missing more than 50% of the numerical ranges present in the real data
--------------------

[trips]:


Creating report: 100%|███████████████████████████████████████████████████████████████████| 4/4 [00:14<00:00,  3.60s/it]


DiagnosticResults:

! The synthetic data is missing more than 10% of the categories present in the real data

DANGER:
x More than 50% of the synthetic rows are copies of the real data
--------------------




C:\programs\conda\envs\tfgpu\lib\site-packages\sdmetrics\reports\single_table\diagnostic_report.py:214: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if pd.Series(errors).notna().sum() > 0:


# SDV Multi-Table Metrics

In [16]:
mt_quality_report = mt_evaluate_quality(
    real_data=real_data_collection,
    synthetic_data=synthetic_data_collection,
    metadata=sdv_metadata)

Creating report:  40%|██████████████████████████▊                                        | 2/5 [00:01<00:02,  1.49it/s]C:\programs\conda\envs\tfgpu\lib\site-packages\sdmetrics\multi_table\multi_single_table.py:51: ConstantInputWarning: The synthetic data in table 'stops', column 'start_date' contains a constant value. Correlation is undefined for constant data.
  warnings.warn(warning.category(message))
Creating report: 100%|███████████████████████████████████████████████████████████████████| 5/5 [00:14<00:00,  2.82s/it]



Overall Quality Score: 55.47%

Properties:
Column Shapes: 57.98%
Column Pair Trends: 42.22%
Parent Child Relationships: 66.2%


In [17]:
mt_diagnostic_report = mt_run_diagnostic(
    real_data=real_data_collection,
    synthetic_data=synthetic_data_collection,
    metadata=sdv_metadata)

Creating report: 100%|███████████████████████████████████████████████████████████████████| 4/4 [02:46<00:00, 41.59s/it]


DiagnosticResults:

! The synthetic data is missing more than 10% of the categories present in the real data
! More than 10% the synthetic data does not follow the min/max boundaries set by the real data

DANGER:
x The synthetic data is missing more than 50% of the numerical ranges present in the real data
x More than 50% of the synthetic rows are copies of the real data


# Join Test

## Join Data

In [18]:
def auto_join(df_list, how='inner'):
    result = df_list[0]
    for df in df_list[1:]:
        shared_columns = list(set(result.columns) & set(df.columns))
        if not shared_columns:
            continue
        result = result.merge(df, on=shared_columns, how=how)
    return result

In [19]:
df_list = [synthetic_data_collection['routes'], synthetic_data_collection['trips']]
fake_routes_trips = auto_join(df_list)

In [20]:
df_list = [synthetic_data_collection['stops'], synthetic_data_collection['stop_times'], synthetic_data_collection['trips']]
fake_stops_stop_times_trips = auto_join(df_list)

In [21]:
df_list = [real_data_collection['routes'], real_data_collection['trips']]
real_routes_trips = auto_join(df_list)

In [22]:
df_list = [real_data_collection['stops'], real_data_collection['stop_times'], real_data_collection['trips']]
real_stops_stop_times_trips = auto_join(df_list)

## Measue # of Rows Abandoned

In [23]:
def leftout_percentage(df1, df2):
    df1_len = len(df1)
    df2_len = len(df2)
    min_ = min(df1_len, df2_len)
    max_ = max(df1_len, df2_len)
    leftout = (((max_ - min_) / max_) + ((max_ - min_) / min_)) / 2
    return leftout

In [24]:
leftout_percentage(fake_routes_trips, real_routes_trips)

0.0

In [25]:
leftout_percentage(fake_stops_stop_times_trips, real_stops_stop_times_trips)

0.0

## CosSim Test For Joined Data

In [26]:
min_ = min(len(fake_routes_trips), len(real_routes_trips))
cos_test_result = []
for i in range(10):
    seed = 123
    cos_test_result.append(cos_test(fake_routes_trips.sample(min_, random_state=seed), real_routes_trips.sample(min_, random_state=seed)))
    seed += 1
cos_test_result = str(round((1-np.mean(cos_test_result))*100, 1)) + '% similar'
cos_test_result

'67.3% similar'

In [27]:
min_ = min(len(fake_stops_stop_times_trips), len(real_stops_stop_times_trips))
cos_test_result = []
for i in range(10):
    seed = 123
    cos_test_result.append(cos_test(fake_stops_stop_times_trips.sample(min_, random_state=seed), real_stops_stop_times_trips.sample(min_, random_state=seed)))
    seed += 1
cos_test_result = str(round((1-np.mean(cos_test_result))*100, 1)) + '% similar'
cos_test_result

'69.7% similar'

## KS Test For Joined Data

In [28]:
min_ = min(len(fake_routes_trips), len(real_routes_trips))
ks_test_result = []
ks_stats = []
p_values = []
for i in range(10):
    seed = 123
    ks_stat, p_value = ks_test(fake_routes_trips.sample(min_, random_state=seed), real_routes_trips.sample(min_, random_state=seed))
    ks_stats.append(ks_stat)
    p_values.append(p_value)
    seed += 1
ks_test_result.append(str(round((1-np.mean(np.mean(ks_stats)))*100, 1)) + '% similar in distribution')
ks_test_result.append(np.mean(p_values))

In [29]:
print(f'statistics: {ks_test_result[0]}')

statistics: 63.4% similar in distribution


In [30]:
min_ = min(len(fake_stops_stop_times_trips), len(real_stops_stop_times_trips))
ks_test_result = []
ks_stats = []
p_values = []
for i in range(10):
    seed = 123
    ks_stat, p_value = ks_test(fake_stops_stop_times_trips.sample(min_, random_state=seed), real_stops_stop_times_trips.sample(min_, random_state=seed))
    ks_stats.append(ks_stat)
    p_values.append(p_value)
    seed += 1
ks_test_result.append(str(round((1-np.mean(np.mean(ks_stats)))*100, 1)) + '% similar in distribution')
ks_test_result.append(np.mean(p_values))

In [31]:
print(f'statistics: {ks_test_result[0]}')

statistics: 60.7% similar in distribution
